In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'V100'
file_name = 'V100'

In [3]:
DATA_DIR = os.path.abspath("../../../prediction_model/data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model/data/")

## Dense Layer

In [4]:
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/0/benchmark_dense__20180904.pkl' %file_name))

for i in range(1,2):
    dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20180904.pkl' %(file_name, i)))])

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.00000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,32.541160,2050.739640,2039.34312,0.0,32.0,0.597766,0.544008,0.518631,0.029012,1.360066e+08,...,0.083000,0.081760,0.085000,0.497600,0.084600,0.083680,0.246040,0.247920,0.256280,0.249760
std,18.507532,1186.159406,1179.77378,0.0,0.0,0.394799,0.369640,0.361900,0.038093,1.584894e+08,...,0.275888,0.274004,0.278887,0.500004,0.278291,0.276913,0.430711,0.431814,0.436587,0.432883
min,1.000000,1.000000,1.00000,0.0,32.0,0.142522,0.140681,0.135016,0.000619,7.280000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,1019.000000,1019.00000,0.0,32.0,0.306616,0.240679,0.226369,0.011219,2.241665e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,32.500000,2056.500000,2038.00000,0.0,32.0,0.476439,0.440071,0.413759,0.019804,7.522635e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,3084.000000,3061.00000,0.0,32.0,0.789076,0.716267,0.687515,0.032654,1.918241e+08,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,64.000000,4095.000000,4095.00000,0.0,32.0,3.072982,2.187743,2.144961,1.002359,1.028516e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [9]:
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/2/benchmark_convolution__20180904.pkl' %file_name))

header = dfConv.columns

for i in range(3,7):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%i/benchmark_convolution__20180904.pkl' %(file_name, i)))])

In [10]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [11]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [12]:
dfConv.describe()

,batchsize,channels_in,channels_out,gpu,kernelsize,matsize,padding,precision,strides,timeUsed_max,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000,50000.000000,50000.000000,50000.0,50000.000000,50000.000000,...,50000.000000,50000.00000,50000.00000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.00000,50000.000000
mean,32.521400,65.959020,66.341420,0.0,3.986080,256.035360,0.501900,32.0,2.499700,8.093568,...,0.082400,0.08420,0.08424,0.500340,0.081880,0.08434,0.252320,0.250440,0.25030,0.246940
std,18.570099,319.530368,320.968619,0.0,2.004965,147.948416,0.500001,0.0,1.114139,13.219683,...,0.274976,0.27769,0.27775,0.500005,0.274185,0.27790,0.434348,0.433271,0.43319,0.431236
min,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,32.0,1.000000,0.148177,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
25%,16.000000,10.000000,10.000000,0.0,2.000000,128.000000,0.000000,32.0,2.000000,1.273990,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
50%,33.000000,19.000000,19.000000,0.0,4.000000,255.000000,1.000000,32.0,2.000000,3.205211,...,0.000000,0.00000,0.00000,1.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
75%,49.000000,38.000000,39.000000,0.0,6.000000,385.000000,1.000000,32.0,3.000000,8.849361,...,0.000000,0.00000,0.00000,1.000000,0.000000,0.00000,1.000000,1.000000,1.00000,0.000000
max,64.000000,9691.000000,9880.000000,0.0,7.000000,512.000000,1.000000,32.0,4.000000,195.232997,...,1.000000,1.00000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000


In [13]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))